In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re
import os

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [6]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [7]:
df['loc']

1     (6835 Fisher Island Dr #6835, Miami Beach, FL ...
2     (9705 Collins Ave #2403n, Bal Harbour, FL 3315...
3     (8024 Fisher Island Dr #8024, Miami Beach, FL ...
4     (1445 16th St, Miami Beach, FL 33139, USA, (25...
5     (300 South Pointe Dr APT 3702, Miami Beach, FL...
6     (15901 Collins Ave APT 2701, Sunny Isles Beach...
7     (6365 Collins Ave APT 3501, Miami Beach, FL 33...
8     (3188 Via Abitare Way #16, Miami, FL 33133, US...
9     (801 N Venetian Dr f, Miami Beach, FL 33139, U...
10    (848 Brickell Key Dr APT 3301, Miami, FL 33131...
Name: loc, dtype: object

In [8]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [9]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Palazzo della Luna\r6835 Fisher Island Dr 6835...,7/29/2022,10/6/2022,27,"$11,000,000.00",3620,"$3,038.67",Liz Hogan,"Compass Florida, LLC",Judith Zeder,Coldwell Banker Realty,Palazzo della Luna 6835 Fisher Island Dr 6835 ...,Palazzo della Luna,6835 Fisher Island Dr 6835 Fisher Island,"(6835 Fisher Island Dr #6835, Miami Beach, FL ...","(25.7627359, -80.14790479999999, 0.0)",25.762736,-80.147905,0.0
2,St Regis\r9705 Collins Ave 2403N\rBal Harbour,4/4/2022,10/7/2022,157,"$10,500,000.00",3556,"$2,952.76",Sylvia Fragos,Great Estates Int'l Realty,Rafael Arias,One Sotheby's Int'l Realty,St Regis 9705 Collins Ave 2403N Bal Harbour,St Regis,9705 Collins Ave 2403N Bal Harbour,"(9705 Collins Ave #2403n, Bal Harbour, FL 3315...","(25.8892229, -80.1228812, 0.0)",25.889223,-80.122881,0.0
3,Fisher Island\r8024 Fisher Island Dr 8024\rMia...,9/1/2022,10/7/2022,18,"$10,400,000.00",3580,"$2,905.03",Karla Abaunza,Luxury Living Realty,Karla Abaunza,Luxury Living Realty,Fisher Island 8024 Fisher Island Dr 8024 Miami...,Fisher Island,8024 Fisher Island Dr 8024 Miami Beach,"(8024 Fisher Island Dr #8024, Miami Beach, FL ...","(25.758099, -80.13760599999999, 0.0)",25.758099,-80.137606,0.0
4,Capri South Beach Tower\r1445 16th St LPH-1\rM...,6/22/2022,10/3/2022,50,"$5,900,000.00",2970,"$1,986.53",Nelson Gonzalez PA,BHHS EWM Realty,Lisa Van Wagenen,Brown Harris Stevens,Capri South Beach Tower 1445 16th St LPH-1 Mia...,Capri South Beach Tower,1445 16th St LPH-1 Miami Beach,"(1445 16th St, Miami Beach, FL 33139, USA, (25...","(25.7892564, -80.1447119, 0.0)",25.789256,-80.144712,0.0
5,Portofino Tower Condo\r300 S Pointe Dr 3702\rM...,5/9/2022,10/7/2022,80,"$2,850,000.00",1960,"$1,454.08",Lisette Vigoa,Brown Harris Stevens,Stacey Margulies,Douglas Elliman,Portofino Tower Condo 300 S Pointe Dr 3702 Mia...,Portofino Tower Condo,300 S Pointe Dr 3702 Miami Beach,"(300 South Pointe Dr APT 3702, Miami Beach, FL...","(25.7677332, -80.1349941, 0.0)",25.767733,-80.134994,0.0
6,Trump Towers\r15901 Collins Ave 2701\rSunny Is...,5/10/2022,10/4/2022,114,"$2,600,000.00",2768,$939.31,Juan Pirela,Viseca Realty LLC,Evgeny Obraztsov,RE/MAX 5 Star Realty,Trump Towers 15901 Collins Ave 2701 Sunny Isle...,Trump Towers,15901 Collins Ave 2701 Sunny Isles Beach,"(15901 Collins Ave APT 2701, Sunny Isles Beach...","(25.9240235, -80.1218071, 0.0)",25.924024,-80.121807,0.0
7,Akoya Condo\r6365 Collins Ave 3501\rMiami Beach,3/9/2022,10/7/2022,153,"$2,150,000.00",1761,"$1,220.90",Nivaldo Soria,South Florida Investments,Glen Janney,Cervera Real Estate Inc.,Akoya Condo 6365 Collins Ave 3501 Miami Beach,Akoya Condo,6365 Collins Ave 3501 Miami Beach,"(6365 Collins Ave APT 3501, Miami Beach, FL 33...","(25.8460388, -80.1196447, 0.0)",25.846039,-80.119645,0.0
8,Via Abitare\r3188 Via Abitare Way 16\rMiami,5/31/2022,10/4/2022,98,"$2,125,000.00",2427,$875.57,Melissa Rubin,"Compass Florida, LLC",Ivan Madera,Fortune Int'l Realty Key Biscayne Inc,Via Abitare 3188 Via Abitare Way 16 Miami,Via Abitare,3188 Via Abitare Way 16 Miami,"(3188 Via Abitare Way #16, Miami, FL 33133, US...","(25.7250044, -80.2427871, 0.0)",25.725004,-80.242787,0.0
9,Venetian Isle Condo\r801 N Venetian Dr PH-F@\r...,7/5/2022,10/5/2022,55,"$2,000,000.00",2500,$800.00,Michael Feuling,Brown Harris Stevens,Michael Feuling,Brown Harris Stevens,Venetian Isle Condo 801 N Venetian Dr PH-F@ Mi...,Venetian Isle Condo,801 N Venetian Dr PH-F@ Miami Beach,"(801 N Venetian Dr f, Miami Beach, FL 33139, U...","(25.7906661, -80.1792675, 0.0)",25.790666,-80.179267,0.0
10,T0hree Tequesta\r848 Brickell Key Dr 3301\rMiami,8/22/2022,10/4/2022,17,"$1,950,000.00",2353,$828.73,Maria Cecilia Cormack,The Keyes Company,Geane Brito,"Compass Florida, LLC",T0hree Tequesta 848 Brickell Key Dr 3301 Miami,T,0hree Tequesta 848 Brickell Key Dr 3301 Miami,"(848

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [10]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only', 'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [11]:
df.at[10,'building_name']=('Three Tequesta')
df.at[10,'address_only']=('848 Brickell Key Drive')

In [12]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only', 'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [13]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('October 2nd - October 8th')

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [20]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [15]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [16]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_100122
